In [29]:
import random
import os
import re
from glob import glob
from tqdm import tqdm
import numpy as np

import pandas as pd
import ast
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from PIL import Image, ImageDraw, _imaging
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import plot_model
from keras.models import Model
from keras import backend as K
from keras import metrics
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from sklearn.model_selection import train_test_split
import tensorflow as tf
from datetime import datetime
from keras.callbacks import History, ModelCheckpoint
history = History()
from keras.models import model_from_json


In [5]:
def showSampleImgs():
    n_samp = 3
    train_df_sample = train_df.sample(n_samp)
    plt.figure(figsize=(4,4))
    for i in range(n_samp):
        draw = train_df_sample.iloc[i]['drawing']
        label = train_df_sample.iloc[i]['word']
        plt.subplot(n_samp,1,i+1)
        for stroke in draw:
            plt.plot(stroke[0], stroke[1], marker='.', color='black')
            plt.title(label)
            plt.axis('off')
    plt.show()    
    
# Show an image from the dataframe
def showImgFromDf(df, index):
    # Show an image
    plt.imshow(df.iloc[index]['img'],cmap='gray')
    plt.title(df.iloc[index]['word'])
    plt.show()
    
def top_3_accuracy(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=3)


def baseline_conv_model(num_filters, num_classes, img_height, img_width):
    model = Sequential()
    model.add(Conv2D(num_filters*1, (5,5), input_shape=(img_height,img_width,1), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(num_filters*2, (5,5), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(num_filters*4, (3,3), activation='relu')) 
#     model.add(MaxPooling2D(pool_size=(2,2)))
#     model.add(Conv2D(num_filters*8, (3,3), activation='relu')) 
    model.add(Dropout(0.2))
    model.add(Flatten())
#     model.add(Dense(2500, activation='relu'))
#     model.add(Dropout(0.2))
    model.add(Dense(2000, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1500, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1000, activation='relu'))
    model.add(Dropout(0.2))
    # model.add(Dense(num_classes, activation='softmax'))
    # model.add(Activation('softmax'))
    model.add(Dense(units=num_classes))
    model.add(Activation(tf.nn.softmax))

    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', top_3_accuracy, 'categorical_crossentropy'])
    return model

def baseline_conv_model_functional_API(num_filters, num_classes, img_height, img_width):
    # Input Layer
    inputLayer = Input(shape=(img_height,img_width,1))
    
    conv1 = Conv2D(num_filters*1, (5,5), activation='relu')(inputLayer)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    dropout1 = Dropout(0.2)(pool1)
    
    conv2 = Conv2D(num_filters*2, (5,5), activation='relu')(dropout1)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    dropout2 = Dropout(0.2)(pool2)
    
    conv3 = Conv2D(num_filters*4, (3,3), activation='relu')(dropout2)
    dropout3 = Dropout(0.2)(conv3)
    
    flat = Flatten()(dropout3)
    
    dense1 = Dense(2000, activation='relu')(flat)
    drop_dense1 = Dropout(0.2)(dense1)
    
    dense2 = Dense(1500, activation='relu')(drop_dense1)
    drop_dense2 = Dropout(0.2)(dense2)
    
    dense3 = Dense(1000, activation='relu')(drop_dense2)
    drop_dense3 = Dropout(0.2)(dense3)
    
    outputLayer = Dense(num_classes, activation=tf.nn.softmax)(drop_dense3)
    
    model = Model(inputs=inputLayer, outputs=outputLayer)
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', top_3_accuracy, 'categorical_crossentropy'])
    return model


In [39]:

# graphviz
# import os
# os.environ["PATH"] += os.pathsep + '/usr/local/lib/python2.7/site-packages/'
# plot_model(model)
# model.summary()
# model

In [6]:
random.seed(111)

# Generate class mappings
fnames = glob('train_simplified/*.csv')
tempDf = pd.DataFrame(columns=pd.read_csv(fnames[0], nrows=1).columns)

for name in fnames:
    data = pd.read_csv(name, nrows=2)
    tempDf = tempDf.append(data)
    
ys = tempDf['word']
    
# Convert class labels from categorical to numerical
unique_classes_list = ys.unique()
map_class_to_numeric = {k: v for v, k in enumerate(unique_classes_list)}
map_numeric_to_class = {v: k for k, v in map_class_to_numeric.iteritems()}

In [7]:
def draw_it(raw_strokes, output_height, output_width):
    raw_strokes = eval(raw_strokes)
    image = Image.new("P", (255,255)
#                       , color=1
            )
    image_draw = ImageDraw.Draw(image)

    for stroke in raw_strokes:
        for i in range(len(stroke[0])-1):

            image_draw.line([stroke[0][i], 
                             stroke[1][i],
                             stroke[0][i+1], 
                             stroke[1][i+1]],
                            fill=255, width=6)
    # Reduce image size
    image = image.resize((output_height,output_width),Image.ANTIALIAS)
    
    return np.array(image)


class DataGenerator(keras.utils.Sequence):
    def __init__(self, num_samples_to_train, batch_size, trainGeneratorDataOffsetPerClass, img_width, img_height):
        self.batch_size = batch_size
        self.num_samples_to_train = num_samples_to_train
        self.img_width = img_width
        self.img_height = img_height
        self.trainGeneratorDataOffsetPerClass = trainGeneratorDataOffsetPerClass
    
    def __len__(self):
        return np.ceil(self.num_samples_to_train / float(self.batch_size))
    
    def __getitem__(self, idx):
        
        numRowsEachDf = int(self.batch_size / float(numClasses))
        skipRows = (idx * numRowsEachDf) + self.trainGeneratorDataOffsetPerClass
        img_height = self.img_height 
        img_width = self.img_width
        
        X, y = DataGenerator._getNextData(numRowsEachDf, skipRows, img_height, img_width)
        
        return X, y
    
    @staticmethod
    def _getNextData(numRowsEachDf, skipRows, img_height, img_width):
        
        fnames = glob('train_simplified/*.csv')
        numClasses = len(fnames)

        # Get n rows from all the csv files and append them into one dataframe
        train_df = pd.DataFrame(columns=pd.read_csv(fnames[0], nrows=1).columns)
        
        for name in fnames:
            if skipRows == 0:
                data = pd.read_csv(name, nrows=numRowsEachDf)
            else:
                data = pd.read_csv(name, nrows=numRowsEachDf, skiprows=range(1,skipRows)) # 0th row is the header, so not skipping that
            # Append the training data of all the classes
            train_df = train_df.append(data)
            
        train_df = train_df.reset_index().drop('index', axis=1)

        # Shuffle all data points
        train_df = train_df.sample(train_df.shape[0])

        # Get only those which were correctly recognized
        train_df = train_df[train_df['recognized'] == True]

        # Convert drawing to images
        train_df['drawing'] = train_df['drawing'].apply(lambda x: draw_it(x, img_height, img_width))

        # Reshape the array
        train_imgArr = np.vstack(train_df['drawing'].values).flatten().reshape((train_df['drawing'].shape[0], img_height, img_width))
    
        # In Keras, the layers used for two-dimensional convolutions expect pixel values with the dimensions [width][height][pixels] for TF.
        # In the case of RGB, the first dimension pixels would be 3 for the red, green and blue components and it would be like having 3 image inputs for every color image. In this case, the pixel values are gray scale, the pixel dimension is set to 1.
        train_X = train_imgArr.reshape(train_imgArr.shape[0], img_height, img_width, 1).astype('float32')
        
        
         # Initialize the y_train
        y_train = train_df['word']
    
        y_train_numeric = y_train.apply(lambda x: map_class_to_numeric[x])

        # convert class vectors to binary class matrices
        train_y = keras.utils.to_categorical(y_train_numeric, numClasses)
        
        return train_X, train_y
    

In [8]:
img_width = img_height = 32

In [9]:
X_test, y_test = DataGenerator._getNextData(100, 90000, img_height, img_width)

In [10]:
# pp = []
batch_size = 1000
numTrainExamplesPerClass = 500
numClasses = 340
num_samples_to_train = (numTrainExamplesPerClass*numClasses)
trainGeneratorDataOffsetPerClass = 0
dataGenerator = DataGenerator(num_samples_to_train, batch_size, trainGeneratorDataOffsetPerClass, img_width, img_height)

In [33]:
# model.summary()

In [40]:
# model = baseline_conv_model(4, 340, 32, 32)
numStartFilters = 8
# model = baseline_conv_model(numStartFilters, numClasses, img_height, img_width)
model = baseline_conv_model_functional_API(numStartFilters, numClasses, img_height, img_width)
model.fit_generator(
        generator=dataGenerator,
        epochs=100,
        verbose=1,
        shuffle=False,
        validation_data=(X_test, y_test)
    
#         ,
#         use_multiprocessing=True,
)
#       steps_per_epoch=(num_training_samples // batch_size),
#       validation_data=my_validation_batch_generator,
#       validation_steps=(num_validation_samples // batch_size),
#       use_multiprocessing=False,
#       workers=16,
#       max_queue_size=32)


Epoch 1/100
170/170 [==============================] - 868s 5s/step - loss: 5.7916 - acc: 0.0108 - top_3_accuracy: 0.0291 - categorical_crossentropy: 5.7916 - val_loss: 5.1251 - val_acc: 0.0357 - val_top_3_accuracy: 0.0875 - val_categorical_crossentropy: 5.1251
Epoch 2/100
170/170 [==============================] - 1223s 7s/step - loss: 4.9127 - acc: 0.0470 - top_3_accuracy: 0.1132 - categorical_crossentropy: 4.9127 - val_loss: 4.5379 - val_acc: 0.0956 - val_top_3_accuracy: 0.1975 - val_categorical_crossentropy: 4.5379
Epoch 3/100
170/170 [==============================] - 1113s 7s/step - loss: 4.4786 - acc: 0.0883 - top_3_accuracy: 0.1917 - categorical_crossentropy: 4.4786 - val_loss: 4.0955 - val_acc: 0.1488 - val_top_3_accuracy: 0.2860 - val_categorical_crossentropy: 4.0955
Epoch 4/100
170/170 [==============================] - 1014s 6s/step - loss: 4.1316 - acc: 0.1277 - top_3_accuracy: 0.2586 - categorical_crossentropy: 4.1316 - val_loss: 3.7205 - val_acc: 0.2001 - val_top_3_accur

170/170 [==============================] - 1089s 6s/step - loss: 2.3356 - acc: 0.4011 - top_3_accuracy: 0.6288 - categorical_crossentropy: 2.3356 - val_loss: 2.4433 - val_acc: 0.4092 - val_top_3_accuracy: 0.6251 - val_categorical_crossentropy: 2.4433
Epoch 33/100
170/170 [==============================] - 1248s 7s/step - loss: 2.3078 - acc: 0.4059 - top_3_accuracy: 0.6350 - categorical_crossentropy: 2.3078 - val_loss: 2.4227 - val_acc: 0.4126 - val_top_3_accuracy: 0.6296 - val_categorical_crossentropy: 2.4227
Epoch 34/100
170/170 [==============================] - 1106s 7s/step - loss: 2.2801 - acc: 0.4094 - top_3_accuracy: 0.6398 - categorical_crossentropy: 2.2801 - val_loss: 2.4307 - val_acc: 0.4101 - val_top_3_accuracy: 0.6302 - val_categorical_crossentropy: 2.4307
Epoch 35/100
170/170 [==============================] - 1019s 6s/step - loss: 2.2764 - acc: 0.4120 - top_3_accuracy: 0.6420 - categorical_crossentropy: 2.2764 - val_loss: 2.4577 - val_acc: 0.4070 - val_top_3_accuracy: 0.6

170/170 [==============================] - 1271s 7s/step - loss: 1.9503 - acc: 0.4727 - top_3_accuracy: 0.7085 - categorical_crossentropy: 1.9503 - val_loss: 2.5417 - val_acc: 0.4085 - val_top_3_accuracy: 0.6221 - val_categorical_crossentropy: 2.5417
Epoch 64/100
170/170 [==============================] - 1447s 9s/step - loss: 1.9322 - acc: 0.4762 - top_3_accuracy: 0.7129 - categorical_crossentropy: 1.9322 - val_loss: 2.5093 - val_acc: 0.4152 - val_top_3_accuracy: 0.6280 - val_categorical_crossentropy: 2.5093
Epoch 65/100
170/170 [==============================] - 1325s 8s/step - loss: 1.9157 - acc: 0.4779 - top_3_accuracy: 0.7160 - categorical_crossentropy: 1.9157 - val_loss: 2.5364 - val_acc: 0.4095 - val_top_3_accuracy: 0.6212 - val_categorical_crossentropy: 2.5364
Epoch 66/100
170/170 [==============================] - 1300s 8s/step - loss: 1.8986 - acc: 0.4837 - top_3_accuracy: 0.7189 - categorical_crossentropy: 1.8986 - val_loss: 2.5976 - val_acc: 0.4012 - val_top_3_accuracy: 0.6

170/170 [==============================] - 1268s 7s/step - loss: 1.7255 - acc: 0.5202 - top_3_accuracy: 0.7540 - categorical_crossentropy: 1.7255 - val_loss: 2.6559 - val_acc: 0.4022 - val_top_3_accuracy: 0.6145 - val_categorical_crossentropy: 2.6559
Epoch 95/100
170/170 [==============================] - 3438s 20s/step - loss: 1.7315 - acc: 0.5188 - top_3_accuracy: 0.7538 - categorical_crossentropy: 1.7315 - val_loss: 2.6396 - val_acc: 0.4062 - val_top_3_accuracy: 0.6129 - val_categorical_crossentropy: 2.6396
Epoch 96/100
170/170 [==============================] - 1413s 8s/step - loss: 1.7350 - acc: 0.5194 - top_3_accuracy: 0.7542 - categorical_crossentropy: 1.7350 - val_loss: 2.6584 - val_acc: 0.3997 - val_top_3_accuracy: 0.6113 - val_categorical_crossentropy: 2.6584
Epoch 97/100
170/170 [==============================] - 1301s 8s/step - loss: 1.7515 - acc: 0.5169 - top_3_accuracy: 0.7491 - categorical_crossentropy: 1.7515 - val_loss: 2.6369 - val_acc: 0.4048 - val_top_3_accuracy: 0.

In [10]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))
# model

CNN Error: 29.11%


In [17]:
# model = 0

In [132]:
# %%time
# # Transform data for CNN
# X_train, X_test, y_train, y_test, imgArr_test, map_class_to_numeric, map_numeric_to_class = CNN_dataPrep(train_df, img_height, img_width)

In [2]:
# num_filters = 2
# num_classes = 340
# continueTrain = False
# if continueTrain == False:
#     model = baseline_conv_model(num_filters, num_classes, img_height, img_width)

# # checkpoint
# filepath="Saved_Models/weights.best.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# callbacks = [checkpoint]

# print(model.summary())

# hist = model.fit(X_train, 
#           y_train, 
#           validation_data=(X_test, y_test),
#           epochs=2, 
#           batch_size=5, 
#           verbose=2,
#           callbacks = callbacks)

In [37]:
# Save the model
# serialize model to JSON
model_json = model.to_json()
with open(("Saved_Models/model_" + str(datetime.today())[:16].replace(" ", "_").replace(":","-") + ".json"), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(("Saved_Models/model_" + str(datetime.today())[:16].replace(" ", "_").replace(":","-") + ".h5"))
print("Saved model to disk")

Saved model to disk


In [25]:
# model = ""

In [24]:
# Load the model back
# load json and create model
json_file = open('Saved_Models/model_2018-11-21_02-44.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("Saved_Models/model_2018-11-21_02-44.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', top_3_accuracy, 'categorical_crossentropy'])
# score = loaded_model.evaluate(X, Y, verbose=0)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
model = loaded_model

Loaded model from disk


# Create Test Data Set

In [38]:
def getTestData(img_height, img_width):
        
        numClasses = 340

        test_df = pd.read_csv('test_simplified.csv')            

        # Convert drawing to images
        test_df['drawing'] = test_df['drawing'].apply(lambda x: draw_it(x, img_height, img_width))

        # Reshape the array
        test_imgArr = np.vstack(test_df['drawing'].values).flatten().reshape((test_df['drawing'].shape[0], img_height, img_width))
    
        # In Keras, the layers used for two-dimensional convolutions expect pixel values with the dimensions [width][height][pixels] for TF.
        # In the case of RGB, the first dimension pixels would be 3 for the red, green and blue components and it would be like having 3 image inputs for every color image. In this case, the pixel values are gray scale, the pixel dimension is set to 1.
        test_X = test_imgArr.reshape(test_imgArr.shape[0], img_height, img_width, 1).astype('float32')
                
        return test_df, test_X

In [39]:
test_df, test_X = getTestData(img_width, img_height)

In [40]:
# Make predictions
preds = model.predict(test_X)
outputDf = test_df.copy()[['key_id']]
outputDf['word'] = ''
map_numeric_to_class_space_normal = map_numeric_to_class.copy()
for key in map_numeric_to_class_space_normal:
    map_numeric_to_class_space_normal[key] = (map_numeric_to_class_space_normal[key].replace(" ", "_"))

for i in tqdm(range(preds.shape[0])):
    outputDf['word'].at[i] = ' '.join(([map_numeric_to_class_space_normal[predClass] for predClass in [tup[1] for tup in sorted(zip(preds[i], range(340)), reverse=True)[:3]]]))

# Create csv
outputDf.to_csv('initial_pred.csv', index=False)

100%|██████████| 112199/112199 [00:22<00:00, 5018.47it/s]
